In [2]:
import pandas as pd
import numpy as np

In [2]:
df_imdb = pd.read_csv('./Dataset/data.tsv',sep='\t')

C:\Users\rafae\AppData\Local\Temp\ipykernel_7128\3123505382.py:1: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df_imdb = pd.read_csv('./Dataset/data.tsv',sep='\t')


In [3]:
df_imdb.shape

(9696517, 9)

In [4]:
df_imdb = df_imdb.loc[(df_imdb.titleType == 'movie'),['genres', 'originalTitle','startYear']]

In [5]:
df_imdb.shape

(638946, 3)

In [6]:
df_imdb.head()

,genres,originalTitle,startYear
8,Romance,Miss Jerry,1894
144,"Documentary,News,Sport",The Corbett-Fitzsimmons Fight,1897
498,\N,Bohemios,1905
570,"Action,Adventure,Biography",The Story of the Kelly Gang,1906
587,Drama,L'enfant prodigue,1907


In [7]:
df_imdb['startYear'] = pd.to_datetime(
    df_imdb['startYear'], 
    format='%Y', 
    exact=True, 
    errors='coerce')

In [8]:
df_imdb['year'] = pd.DatetimeIndex(df_imdb['startYear']).year

In [9]:
df_imdb.drop(columns='startYear', inplace=True)

In [10]:
df_imdb['year'].value_counts().sort_index()

1894.0     1
1896.0     1
1897.0     2
1898.0     7
1899.0     8
          ..
2026.0    18
2027.0     7
2028.0     1
2029.0     3
2030.0     1
Name: year, Length: 136, dtype: int64

In [11]:
df_imdb = df_imdb.loc[df_imdb.year > 2017,['genres', 'originalTitle','year']]
df_imdb = df_imdb.loc[df_imdb.year < 2023,['genres', 'originalTitle','year']]

In [12]:
df_imdb['year'].value_counts().sort_index()

2018.0    18735
2019.0    18487
2020.0    15908
2021.0    17985
2022.0    19018
Name: year, dtype: int64

In [13]:
df_imdb['genres'] = df_imdb['genres'].replace('\\N', np.NaN)
df_imdb['genres'] = df_imdb['genres'].str.replace(',', ' ')

In [56]:
df_imdb.isna().sum()

genres           0
originalTitle    0
year             0
dtype: int64

In [57]:
df_imdb = df_imdb.dropna()

In [58]:
df_imdb['year'].value_counts().sort_index()

2018.0    17735
2019.0    17439
2020.0    15001
2021.0    16965
2022.0    18087
Name: year, dtype: int64

In [59]:
df_imdb.to_csv('./Dataset/processed_dataset/IMDB_dataframe.csv',index=False)

In [60]:
from tmdbv3api import TMDb
import json
import requests
tmdb = TMDb()
tmdb.api_key = '6991ecf53898596bfe405ab83d0524f1'

In [63]:
from tmdbv3api import Movie
tmdb_movie = Movie()
def get_director(x):
    crew = []
    try:
        result = tmdb_movie.search(x)
        movie_id = result[0].id
        response = requests.get('https://api.themoviedb.org/3/movie/{}/credits?api_key={}'.format(movie_id,tmdb.api_key))
        data_json = response.json()
        if data_json['crew']:
            crew_str = " " 
            for i in range(0,len(data_json['crew'])):
                if(data_json['crew'][i]['job'] == 'Director'):
                    crew.append(data_json['crew'][i]['name'])
            return crew_str.join(crew)
        else:
            np.NaN
    except Exception:
        print(x)                   

In [61]:
df_2018 = df_imdb.loc[df_imdb.year == 2018,['genres', 'originalTitle','year']]
df_2018

,genres,originalTitle,year
67669,Drama,The Other Side of the Wind,2018.0
108911,Comedy,A Thin Life,2018.0
185281,Drama,Reverse Heaven,2018.0
264779,Action Drama,To Chase a Million,2018.0
314833,Drama Family,Aamaar Bhuvan,2018.0
...,...,...,...
9693290,Drama Family,Hot Scent,2018.0
9693635,Documentary,"Luz, Câmera e Barreto",2018.0
9695525,Documentary,9/11: Escape from the Towers,2018.0
9695534,Comedy,Wien is 't Hof van Commerce,2018.0


In [64]:
df_2018['director_name'] = df_2018['originalTitle'].map(lambda x: get_director(str(x)))

Amsteroid
RxCannabis: A Freedom Tale
Lusitanian Ghosts
Feeling of revulsion
Weltweite Freundschaften
The Stateless Diplomat
Skvoznyye Shagi
Lost Einstein
Emiliano Queiroz: o Ator em sua Essência
Anuttara
A Song In The Storm
A Song in the Storm
1200+ Dir
Saboa
Nationalmuseum - ljuset återvänder
Latinos en el corredor de la muerte
El mercado de la inocencia
Levaditi - Agada al Miriam Yalan-Shteklis
El Hombre de la Montaña
Premak Basaat
Paranaue: The Documentary
A Idade da Água
Zwitscherland
Durs Appenzellerland
Pur laine
Miró contra Miró
Tian Shang De Hai Zi
The Age of Sebastian
Crossing the river of life
Jeremiah's Woods
Ibed Fogel et Fogel
Das Bürgermeistermädchen
Rotenberg
La fin da la val l'è mia la fin dal mund
Second Hand Store Horror Tales
Antropofaga Makumba
Taste of Friend Daughters
Paracelso
Une vie d'Annette
Brother's Good Wife
Legends Never Die: The Sandlot Story
Shimanto -The Peaceful Death
Restoring Tomorrow: Rise Above
Biserta. Storia a spirale
Panorama: Jamming to the Top

Port of Destiny: Peace
Embrapi - Um Sonho Interrompido
100 hronia SEI
Vo mrake
Jerusalem or Bust
Ersilio - Il Cardinal Tonini, i media come pulpito
With No Regrets: A Dancin' Man's Journey
Wo shi hao ren
Werewolf Fatal
Changan demon talk
Polar Lady Tied Up Beauty Wife Sono Six
Bound and Beautiful Married Woman Wet Rope Lust
77-Star Wars-17
Puisque nous mourrons
First Impression 130 Junmai: Birth of A Beautiful Pure Pretty Girl
Kiss Crazed Covered In Gooey Saliva 4 Scenes Act. 03 Naughty Lips That Are More Sensitive Than Her Pussy
Les Jours Inespérés
Mehandi Laga Ke Rakhna 2
Karam Yug
Awara Balam
Sahara, la fureur et l'oubli
Mai Re Hamara Uhe Laiki Chahi
Balam Ji Love You
Sarkuft
The Bad Ex-girlfriends
Half Court Hal
Portal dos Sonhos
You Make My Life Beautiful
The Legend of Liuyi
A Mysterious Date
1968 - Sport & Revolution
Coronado - Tagged for Life
Le Voyage Mexicain
Uncle Fun: You're the One
Celtic Pilgrimage:
Glossodetes
Nane Kradod Kampaeng
Pathis Gang
B'hike n BASE
The Brigade - t

Dhanyabad
Hypnose op recept
Verdacht: Hoe vaak wordt u staande gehouden?
Comrade Shanta Ranade
Pionierzy
Kitanaka suketchi
Solo: The Evolution of the One Man Band
Genocide in Twente
Schimmenspel: Poetins onzichtbare oorlog
Hallo met Kyoko
Onze manier van leven
Waarheen leidt de weg?
Lief en leed op Parc Beaugarde
The making of oom Jo
ShobBahika
Jiazú. Láodòng. Syoping.
War Retroactive
Barclays English Premier League 2017-18 Season in Review
Amatörle karsi karsiya
Pelota Pulpo
To the Ends of the Earth: A Portrait of Jewish San Diego
The Perfect Match - The Sport of Polo in the US
China's Swan Lake
Vuelta a la Izquierda
Damnificados. Los negocios de la reconstrucción
Liverpool Legends
Apocalypse Society
Song of Love I Can't Help
Bravo 2: Machinima Heroes
Thinking Nietzsche
Messi: All 600 Goals in His Career
Serevi - Fiji's King of Sevens
The True Colors of US Universities
Zayed's Antarctic Lights
Backroads MD
Cristiano Ronaldo All 450 Goals for Real Madrid
Same Music, Different People
Li

Monika Brugger, entretien avec Laurence Verdier
Saaguva Daariyalli
Empire of ballet
Little Minds on the Prairie
Octubre 23
Det Goda Landet'
Det goda landet: Arbetsförmedlarna i Göteborg
Det goda landet: Major Nyström
Det goda landet: Läraren Maja
Det goda landet: Kommuntjänstemännen i Falun
Det goda landet: Folkdansaren Lena
Le bruit de nos silences
Ruy Jervis d' Athouguia - Um Moderno por Descobrir
Rifftrax: A Talking Cat!?!
Legend of a Hacker
Tamburica: The Sound of a Landscape
Un Dial de Historias
In Tune: The Ben Tucker Story
Patriot: War in Ukraine
On a beau tuer les hirondelles
Actress for Hire
Guiting - Live!
Cinemagar Talk Show
What Is Food the Food Film
The Run from the End
Storie di vita: Un amore morboso
Avalin Emza Baraie Ra'na
Wild Taiwan
Masakali
Zani Ba Arabeye Choobi
Charlie Dar Tehran
True North, the Story of ASRC
Running for Film - My Run Across America
Un chez-soi d'abord
Shankhanaad
Shri Haasanamba Mahime
Yogi Duniya
Dagabaaz Piritiya
Walking for Genna
RN 193
Matt R

Mia olonyhtia gia ton Saixpir
Pontos: Mnimes stin omihli tou parelthontos
Puknatini
To taxidi tou Orfea
Saulo Gomes: O Repórter de Chico Xavier
Lala Kon
Ik Onkaar
Truth of Change
Broken Walls
PriPara & Kiratto PriChan Movie: Sparkling Memorial Live
Larissa - The Lost Romanov?
Tsumetai Kisu
Di Xue De Hong Du Juan
Cenizas en un Lienzo
Xavi Sí, Xavi No
Khandar
Le Singulier destin du Sergeant York
Almost a Virgin
Glavnyy grek Rossiyskoy imperii
Christmas: The Story Behind the Traditions
No Man Shall Protect Us: The Hidden History of the Suffragette Bodyguards
Orainaldian
Honey and Bunny the Secret Formula
Achille Castiglioni. Tutto con un niente
Marital Bruise
Living Legend - Denroy Morgan
Recession Blues
100 años en blanco y verde
Grand Central Terminal, la gare des étoiles
Taming Winter
Le Caire, porte d'Égypte, porte du monde
Vladivostok, l'aube se lève sur la Russie
Krajina s vônou medu
Living Legend: Commissioner Gordon
NEWMUSEUM(S). Stories of Company, Archives and Museums
Dr Duncan


...

Wax in the City
The Black Box of the Art Business
Wee Little Horror Movie
Abalos: Una Historia de 5 Hermanos
Echos einer Ära: Simon Rattle & die Berliner Philharmoniker
Addict 2018
Cadlag: Votivkirche
Woken Terror
L'important, c'est la rose
The Night Of Prisms
Johnny do Brasil *
Electronic Crack Film
Il gatto con gli occhi rossi
Deyo Wadhaiyan Sada Vi Vyah Hogeya
Reinventing Power: America's Renewable Energy Boom
Big Election
Zero Distance - Renaissance Dam
Paranormal: Tales of the Unknown
The Blue Redemption
Going West or 15 Years in LA
Pentru Romania
S' afti ti hora kaneis den ixere na klaiei
We Are One - blood drum spirit
Jean Vanloo: King of Clubs
Imeool wihan haeng jingok
Honky Tonk Moonlight
El-Ablah Tam-Tam
Forcing Function
From the Ripple Tank
The Age Demanded
AOP vs. The World
Channeling Hamilton
Hymn to the layman
Never trouble trouble until trouble troubles you
Building a Dancer
Deo pensyeon
Thravsmata
Patrida apo marmaro
Nikos Mamagakis: Logos teleftaios
Black Girl Magic th

Nivuru
Hasiru Ribbon
Divya Mani
The Shoah in France, turning point of 1942
Leeyaanns
Bhai Jaita
Divin Bonheur
Les Petites Mamans - Mamatchki
Campus Da Challenge
Pushpak Vimaan
Dostigiri
Huang Fei Hong: Nan bei ying xiong
The Maurits Kiek Story
The Culture of Ignorance
Untitled Monkees Project
Big Band's Last Stand
Ser O Estar Redux
The Cast Squid of a Lost Character
Instagramshizza
Just Get My Name Right
This Business of Autism
Pastor Greg's Reboot - Hope for Christmas
Deal with It! The Snail Heads Movie
L'Odeur du Gaz
An Unwanted Guest
Relax Satya
Wish You Weren't Here: The Dark Side of Roger Waters
Tango on the Campania
Dil Hona Chahida Jawan
Non cercarti fuori
Maut Phir Aayegi
Pardon my genes
Sesat: Yang Sudah Pergi Jangan Dipanggil Lagi
Shuluq - Vento di mezzogiorno
It Was Supposed to Be a Love Story
Dead Musician Fan Club
space.games.film
Tales of the Baron
Pure Art, the Work of Franklin Williams
Heiler ohne Grenzen - Eine Reise in die Dankbarkeit
El-Badlah
The Social Deconstructi

Sankt Lars sjukhus - där satt ju dårarna
Igniting Impact
Silent Love Stories: Heart Beat
Ha Ling Peak
Wei ze zhi lu
The President's Bodyguard
Naoto Inti Raimi boukenki 2: Tabibiuta daiyarî - Kohen
Chu Oi, Dung Lay Me con
Demain Genève
El Terremoto del Tiempo
Re-sound
448 Journey of the Haast Eagle
Angelena: Heart Of The Matter
Ek Raadha Ek Meera
Conversazioni atomiche
Baco Becce
Apokalypsi akoma: Apodohi klironomias
The Flaws of Copyright Infringement Laws
Der Film der nie gemacht wurde
Being Mario Götze
Frampton: Return of the Jackal
Cartas a Elizabeth
Arwah Tumbal Nyai the Trilogy: Part Arwah
Wind, Rain, & Time
Shaheed- E- Aazam
Joiun voima
50 mi zhi lian
Mausam Ikrar Ke Do Pal Pyar Ke
The Grand Lady of Iran
Father at Fifty
Chi ha paura di Giuliano?
The Glorification of Gwen
Larmes en cendre
Chuck, Lost Object Found
Mean Bean
Ya, Ty, On, Ona
Tales of Resistance - The Battle of the Newbury Bypass
Confirmation 101
Disparo A Un Caballo Oscuro
Field of Blackbirds
What Becomes of Us
The I

Ecuador, My Power into the Constitution?
¡OJALÁ! - Puerto Rico Rebuilds
St. Joseph Freinademetz: The First Saint to Ever Serve in Hong Kong
Johnny Taylor: Bummin' with the Devil
Scarlet Cay
Man Sitting in Chair
Las Brujas de E'lente
Ashleel Udyog Mitra Mandal
Hand of the Master: The Art and Life of Dom Gregory De Wit
Revolutionizing Dementia Care
Veronica's Wish
Deadly Sexy
Saahasi Makkalu
Kannada Deshadol
Gvana Yajna
In Defence of Sacred Images
Pending love
Um Milagre Todos os Dias
Desa dimmari
Thanthrika
Galli bakery
Graines de guerre
Kathale kone
You 5 ge jie jie de wo jiu zhu ding yao dan shen le a
PK Lele a Salesman
Like: A Documentary About the Impact of Social Media on Our Lives
Dark Date
They Call Me Busker
La última toma
Game of Phones
Plastic Blastic
Don't Stare at Me!
Stanley's War
Uri Zohar Khozer
Franco on Trial: Spain in the focus of international justice
Design Space for Space Design
A Different Business
La catedral de las 6 cuerdas
No Man's Land - Expedition Antarctica


The Bachelors of Broken Hill Farm
Chelpa Ferro
Once we are dancing
A Fanboys Tale
Rockumentary: Evolution of Indian Rock
Meng de bei hou
Star Wars: The Fans Vs Disney
The Swan Princess (Dangina)
Sur le bout de la langue
Bemidji
Once it Was a Land Called Pomerania
Lima Screams
El Navío y sus Faros
Vill du dansa?
Kenneth Tindall's Casanova
The Tides of Fate
The Dawn Never Comes
Eiga Okaasan to Issho hajimeteno daibouken
Jackie and the Genie
Zuwanderung nach Siebenbürgen. Erfolgsgeschichten (Dreams come true in Transylvania)
Pigs Of Paradise
A szabadság bolond körei
Ethniki Vivliothiki: To mellon tis istorias mas
The Living Thames
The Fabulous Trickster
MilzaMan
Il sequestro Moro - Gli altri testimoni
The Girl Without a Phone: A Beauty & the Beast Story
Tridevyatyy uchastok
Code Geass: Lelouch of the Rebellion Episode II
Bimba - Aa Tombatthu Nimishagalu (Bimba - Those Ninety Minutes)
A Smartphone User's Guide to Etiquette
Ultramar - The Documentary
44 Days
Le théâtre nomade
Cinema i Utopi

In [67]:
df_2018.to_csv('./Dataset/processed_dataset/df_2018_dataset.csv',index=False)

In [77]:
df_2019 = pd.read_csv('./Dataset/processed_dataset/IMDB_dataframe.csv',sep='\t')

In [78]:
df_2019 = df_imdb.loc[df_imdb.year == 2019,['genres', 'originalTitle','year']]
df_2019

,genres,originalTitle,year
11636,Action Crime,Tötet nicht mehr,2019.0
152868,Fantasy,Har Har Mahadev,2019.0
264380,Animation,The Fetishist,2019.0
325621,Drama,Wo men fu fu zhi jian,2019.0
369635,Crime Drama Mystery,Motherless Brooklyn,2019.0
...,...,...,...
9696050,Comedy Drama Fantasy,My Girlfriend is a Wizard,2019.0
9696186,Documentary,Drømmeland,2019.0
9696190,Drama,O Ensaio,2019.0
9696315,Adventure History War,Hong xing zhao yao Zhong guo,2019.0


In [79]:
df_2019['year'].value_counts().sort_index()

2019.0    17439
Name: year, dtype: int64

In [80]:
df_2019['director_name'] = df_2019['originalTitle'].map(lambda x: get_director(str(x)))

Wo men fu fu zhi jian
Disrupted Land
Jadugar 2
Ali the Blind Boxer
In Their Own Words, Billy the Kid & The Lincoln County War
Barof 2019
He's My Daddy Too
Sifir: Etkisiz Eleman
On the Trail of Bigfoot: The Legend & The Search
In Our Blood - El Comienzo de Todo, Àlex Crivillé
In Our Blood - Sin Límite, Marc Márquez
Encrypted: The Cyber Crime
Rocky Mountains Encounter
The Soul of Soho
Zawierucha in Hollywood
RC Propbusters: Untold
Jai Chhathi Maa
919 Lakeview Drive
Forest of the Dead Sharks
I Watched TV Show and Died from Blood
100 Kada 10
Garud Puran
Papers 14: Purge of the Papers
Sucker for Love
Entre toits et moi
Under 18 - Storie di sogni periferici
The Macao memory
67 49 32 Kuzey
Satyarthi
The Elusive Elseblades
Kanchani Aaj Bhi Hai
Brightness of Noon: The Intersect of Faith, Immigration and Refugees Part 2
O Xanthos Magos: Otan to show teleiosei
One God, Three Religions
Death of a Traveling Life Insurance Salesman
Art in the City, 2 hour abridged
Bring Me the Head of Jade Brooks
Ml

Wallet Sized Photo
Between Scylla and Charybdis
Saving the West Side (Again)
Trikonam
Shi tu xing zhe 2: Die ying xing dong
Revenant Revolver
La clase de al lado
Thanthai Sol Mikka Manthiram Illai
Mudivilla Punnagai
Buganda Royal Music Revival
Code & Response
Cinecittà - I mestieri del cinema Bernardo Bertolucci: No End Travelling
Doctor Who Banished
Whistleblower 2044
Khanana
Vedhamanavan
Luo Yang's Girls
Dhigil @ Home Stay
Khananam
Traya
Novembernalli Naanu Avalu
Reisha Perlmutter: Lost and Found
I know Hamlet
La Flor de Taboga
The Ultimate Ghostbusters
Galaxy 360
La famille Ouellette feat. Greg Beaudin & Judith Little D: Satin
Frederic Chopin and la Nouvelle Athènes
Elevator: a deadly lift experience
Göding
Artist.Activist.Administrator
Kondaeel
La pérdida del cuerpo
Tasweerein
Sasquatch Speaks
Eorin uiroein
Decàleg Ferran. Vida i poesia de Jaume Ferran
Sadho
Raffaele Sollecito
#MeAfterToo
Soc perquè som
Khatemeh
The City of Las Vegas: The Early Years
79 Parts: Director's Cut
Remnan

Bokeh and Tsukkomi
Eish Hayatak
Byeol-ui jeong-won
One Night in Al Aqsa
Mikoshi Guy
Dirección única
Wildlife Symphony, Under the Sea and The Amazon Rainforest
Les pépites du fleuve
Estherleon: Tears of stone
Wir waren nicht Menschen - Geschichten vom Überleben
Tiao wu ba! Da xiang
Legacy of the Blade
Santa Soja
Sherlock Holmes: Bound to the Dark
The Ancestors' Memories
Potuto: A timeless story
A Hedwig Reunion
The Comeback - a tale of irrelevance
Condottiere
Jeontoo
Ha'Kha'ver Dov
Don't Date Your Sister
Lectura Ovidii
Daniel, Debra, Leslie (and You?)
The Man History Forgot
Not About a Title
Corruptissima
Golmaal Love
Has Jhan Pagli Fas Jabe
Balkan Blood
Velocidad 22: Gira Bolivia-Peru (y Jujuy!)
At beingining of dessert
Eshche odin prekrasnyy den
I Hope I Get It!: Part 2
The Light Between Two Towers
Bad Water. Small Town. Deaf Ears. Everything You Need to Know About PFAS, But Don't Know How to Ask
Waiting for Farajallah
Los que buscamos
A People's Soundtrack
J'ai le Syndrome d'Angelman

Stalingrad: The Campaign
From Nowhere: The Story of the Vaughan Brothers
The Passion of Jekyll & Hyde
Pyro, Gators & the Devil - John Eggett on Fear No Evil
Stefan Arngrim - On Giants and Feeding Darkness
Metamorphosis Centenary Edition
Rhoads -Kelle & Kathy Rhoads-Mini Documentary-The Metal Voice
Zé Pedro Rock'n'Roll
The Cleveland Chronicles
Queens of Beer
Yuyeolui eumagaelbeom
Preeti Irabaaradhey
Shigunaru 100
Inde, la magie des cerfs-volants
Wizards of the North - The First Battle
Ayushmanbava
The Horn of Fear
Jack Stone: End of the Line
Welcome to Switzer Land
Dhehi- Her soul never surrendered
Ljudi koji su sadili drvece
Milo's Monster School Vlog 1 - Super Simple
Milo's Monster School Vlog 2 - Super Simple
Muistoseurat 3
Shinka no toki
Black and Aliens
Murders in the Forest of the Dead Sharks
Take a Chance: A Musical
Grrrb
Sing Along With Tobee 3 - Super Simple
Residência em Risco
Bukovina - Tara Oamenilor Fagi
Bekkigondu Mooguthi
Dheera Kannadiga
Name Plate
Char A 420
March Randa

Nirahua Chalal London
Qin mi lü xing
Dead and Deleted The Ted Giles Story
Brexit: The Will of the People
The Santa Cause
The Bachelor Party: It's a Wonderful Lifestyle (The Rise of Showstopper)
Premasura
Los Verbenas
Vazhga Vivasayi
Peopekteu maen
Yangjamoolrihak
Murkal Estate
Jacob & Wilhelm - Weltenwandler
WW-II, Film Clips from Forgotten Movies
Tegzas 2
'Kakure bicchi' yattemashita.
Elparvar
Sota una nit d'estels
Test City, USA
Ish HaCherev: Gabriel Moked
Shah Kosh
Heroes of BAF
By Diesel Luchansky
Kilo Flight
Be Witness Documentary
Gajang Botongui Yeonae
MULT v kino. Vypusk # 91
Fracking the Contract
Tamazight transition
Gundam 40th Anniversary Celebration: Chars Counterattack
The last midnight chapter 3: Wendingo
Leaving Neverland: The Aftermath
James Hubbell - Between Heaven and Earth
Humanitarian: The Real Michael Jackson
Foi no Carnaval que Passou (Ashes)
Mystery Weekend Package
Luv Gone Wrong
Just Another Day in Paradise
The Flying Lion
Martial Arts History Museum: Michelle Ma

...

Mitchaku 180-nichi naze moriyasu hajime wa nihondaihyo kantokuna no ka
Athletes' Soul - One and Only Japanese National Football Team Ritsu Doan
Iêmen: Um País Esquecido
Cabeça do Cachorro
Heróis da Lama
Stanley Kubrick, de l'autre côté de l'arc-en-ciel
Une Histoire Moderne De La Pop Culture
Death to the N-Word
High School Soccer European Expedition 2019
Sports x Human - Take Me to the End of Soccer Naohiro Takahara
Gilead's Protocol
Daydreaming in the Hood
Ramón Menéndez Pidal, la historia oculta en las palabras
Cashless India
Ojos que no ven... Movimiento Espartaco
The Newport Bridge: A Rhode Island Icon
More Than Just 4 Letters
Pages in Between
Table of Men
Fantasmagorías
Manoshangko: (Koshtipathor 2)
Qurratu Aiyun: Je Jibon Juray Noyon
Modern Vidya Niketan Schools
#QueRicoPaís
Grumpy Grannies
Lluvia y ceniza
All Güt Things: The Life and Death of Texas German
500 Years of Music at Chambord Castle
Una donna. Poco più di un nome
Vettaiyan
Matusepäevikud
Taunting Faith
Actors of God
Der

Inteonhyeongsa ogyeonsik
Power of Pearl
Punarvasu Nakshatra Mithuna Raashi
Un'Odissea e' Rimas Nobas
Khane Bi Ban
Channel 309 (I-III)
Saraswati - EK Khoj
Finnish American Rag Rug Weavers
Passing It On: Ballymun Kickhams
The Sisyphus Tears by MemoQuartet/the Director's Cut version
Shadow of Self
Geldzahler Does Geldzahler
Khat-e Esteva
Traición en la bahia
A ellas les gustan mayores
Jonarobi
Yooper Creoles: Finnish Music in Michigan's Copper Country
Kudi: My Name Is Ramadan 2
Finnish American Chip Woodcarvers
Pandora Palace
Melvin Kangas: Kantele Man
Gheddis
Upon a Promise
Reckless Two
Love and a Crowd
Scorned Queen
Le Dernier d'Entre Nous
Pa Pa Wadi See Yin Khan
Kha Yay
Palpitation in the Breeze
Alero's Dilemma
Monsordecor
Kodagra Sipayi
Glazami detej
Le gardien du temple
Nansins
Yiannimize Grand Tour
The Way to Dawn
Omoniyun
Love U Dulhin
Harf Akhar
Cavad Xan
Skate Pelas Sombras
Totally Elvis: Image of the King Concert
Code Blue: Resuscitating Rural Surgery
The Essex Warriors
Gelateri

Teddy Bears Live Forever
D'Sory
Let There Be Drums: The Movie
Keep It in the Streets
Surviving Entertainment
Nación de Muchachos
Awani: A colonial history of women
Science & Orthodoxy Around the World
Gekijouban Fate/Stay Night: Heaven's Feel - II. Lost Butterfly
GohRaalhu
Upeksha
Brash Girls Club
Turkish Rose
Cazadores de Orquídeas
Baila con el corazón
Na-reul chaj-a-jwo
The Elevator Game
Waiting for Bardot
A Life to Choose
A Shot for Justice
Indonesia Startup
Dreams on Wheels
In pencil? Omar Galliani
Mia: Love Never Dies
Vse o nas
The Last Bell ... The First of Many
Anime notturne
Herr Berger sucht einen Sohn: The Making of
Being Golden
Finding the Night of the Dead
The Horror of Making My Film
Breathe Breana
My View: Clem Burke
Les portes du ciel
Valentin, son of Europe
So duk 2: Tin dei duei kuet
Anohi no orugan
Kaaneen: A Secret Search
The Tiny House Movie
Li'l Mayne and the Knuckleheads
Leaps of Faiths
Closing Costs
Final Session
Ode to Paz
Aasra
Under the Jazz: Circulation.Forma

Suburban Legends: Life on Rainbow Road
Stâtoappu gâruzu
Bornodi Bhotiai: Love, by the River
Dublajla Düzelir
1960 1970 Turk Fantastik Sinemasinda Ses ve Görüntü Efekt Uretimi
Inside this Peace
The Things We Lose
Nu gui ai shang shi
Self-Taught: Life Stories from Self-Directed Learners
Prementha Panichese Narayana
Dokyala Shot
Humans of Our World: The Journey
We Live in Public/Zivimo u javnosti
The Vikings of Oak Island
The Aamazyn Killer
Hu Narendra Modi Banva Mangu Chu
Hikinige: Saikô no saiaku na hi
Hell's Gift
HK Beauties
O Homem-Pykante - Diálogos kom Pimenta
Da Pinche Code II
75 South
Take Your Mark Banu
Dis-sociated
Auf und Davon - ein Jahr danach
Savasin Efsaneleri
Mono Deux
Maryam of Tsyon - Cap I Escape to Ephesus
Pagal Kar Diya Toone
EcoSPEARS: The Journey to Clean America's Waterways
Denizlerimizin Hukukcu Kaptani: Gunduz Aybay
Surviving Birkenau: The Dr. Susan Spatz Story
Lafange Nawab
When the Sun Fades Away
Holiday Unplugged
Joe KroArt's World Center
Aamhi Befikar
Every S

Chi o sû nendo: Hasei
Murphy's Law: Ghanoone Morfi
Love Struck Sick
The Art of Directing: David Lean
Seytanla 24 saat
Premrang
Kometa: Movie
Windcatcher Dream
99 Luftballons und ein bisschen Frieden - Der tolle Schlagerabend
Driven By Loss
Just Some Kids
Papers XIII: Blood Currency
27 Rue de Fleurus
Sugartown: Gia mia houfta psifous
Kathleen Kilbane - In the Presence of a Holy Child
I Saw the Hosts of the Dead
The Informants: Who Killed Nicole?
The Unknown Life of Antarctica
Trafficked--Africa
Affliction or (Halcyon)
Plastic Crack Documentary
Tinanjali, a song of Life
The Story of a Panty
Eterno Quini
Charmaine
An Ungodly Tie
Each One Teach One
Gui men shi san zhen
Porvenir, Texas
The End of the World - Episode 5: Civil War
Traveling Landscape
Kuronosu Jôntâ no densetsu
La sonrisa de Krahe
Liquor House Comedy 2
Ta roda, ta triantafylla (To bloko tis Kalamarias)
The Elephant King Adventures
Vingo
One Love 2 Story
Militia Man
Comunismo futuro
Brush Mileage
Jiwan Kanda Ki Phool
Subha Love

In [89]:
df_2019.isna().sum()

genres              0
originalTitle       0
year                0
director_name    5691
dtype: int64

In [91]:
df_2019 = df_2019.dropna()
df_2019.isna().sum()

genres           0
originalTitle    0
year             0
director_name    0
dtype: int64

In [94]:
df_2019.shape

(11748, 4)

In [92]:
df_2019.to_csv('./Dataset/processed_dataset/2019_dataset.csv',index=False)

In [96]:
df_2020 = pd.read_csv('./Dataset/processed_dataset/IMDB_dataframe.csv',sep=',')
df_2020

,genres,originalTitle,year
0,Crime Drama Thriller,Back Roads,2018.0
1,Drama,Mistero di un Impiegato,2019.0
2,Documentary,Disrupted Land,2019.0
3,Drama,Pattabhiraman,2019.0
4,Documentary Music,Lusitanian Ghosts,2018.0
...,...,...,...
68946,Mystery,Zwtral,2021.0
68947,Drama Music,Zycie w blocie sie zloci,2022.0
68948,Thriller,TRUE,2021.0
68949,Adventure History War,Hong xing zhao yao Zhong guo,2019.0


In [97]:
df_2020 = df_2020.loc[df_2020.year == 2020,['genres', 'originalTitle','year']]
df_2020

,genres,originalTitle,year
13,Drama,1969,2020.0
92,Drama Fantasy Sci-Fi,2050,2020.0
100,Mystery Sci-Fi,2067,2020.0
223,Comedy,$30 Worth of Funny,2020.0
244,Comedy Romance Sci-Fi,(NE) Idealnyy muzhchina,2020.0
...,...,...,...
68796,Adventure,Zorro: New Orleans,2020.0
68805,Drama Romance,Zovi menya Drozd,2020.0
68817,Drama Thriller,Zu den Sternen,2020.0
68818,Action,Zuìran de quantou,2020.0


In [98]:
df_2020['year'].value_counts().sort_index()

2020.0    9570
Name: year, dtype: int64

In [99]:
df_2020['director_name'] = df_2020['originalTitle'].map(lambda x: get_director(str(x)))

$30 Worth of Funny
(Ne)sluchayniye istorii
_Titans
10 Jahre später
13 Tribute of Love
2 Ekre
2020 Covid 19 Zombi Masacre
2188 a lab of magic film
24 Hours in Nowhere
28 Degree Celsius
3 Days 4 Nights in Bihar
3 Shyaane
3.14...
3:30PM
30 Days To Kick Ass Love
3BHK Movie 2020
4 m@xime
4.04 (April 4th)
40'N Still Single
5 Adi 7 Angula
51: After the Raid
5Ws
6 Rounds of Chloë
66 Brews
7 Bxys kéath
72hrs the Hague: Viral
8 Semanas De Aislamiento
8-ka de shinda kaijû no 12-nichi no monogatari
9 povesti de dragoste si ura in izolare
90 Minutes TV Show
90ème minute
A Bachelor Next Door Suffered by A Sneaky Breastfeeding
A Beautiful Distraction
A Branch to Roost
A Christmas Night:1944
A death in Peking
A Dragon Christmas
A Dream Called Death
A Familiar Lie
A Family Home Going
A fool's errand
A Frantic Love 2
A Free Can Gangster
A Full House of Empty Rooms
A Gambler's Guide to Dying
A Girl's Deranged Goodbye
A Good Year in Metaphorland
A Hostile Nation
A House in Your Neighborhood
A Lost Song: T

ClippiHoster
Clock Tower the Movie
Cock Tail
Cold Calm
Collapse 2045
Color of Vacancy
Colours of Deceit
Comando Central
Comedy Bad Boys 2020
Comedy Hustle: The Chitlin' Circuit
Comedy in Color
Comedy Koduran
Comedy Sketches
Comic-Con Film School
Coming Through America
Comment allez vous? Je suis épuisé
Con Actors
Concrete Rose
Conflict: A Star Wars Story
Conflicts of Green
Connected Demons
Conscious Consequences
Conservators IV: Fracture
Conspiracy Child
Conversations - An Original Virtual Play
Cops - Una banda di poliziotti
Cor-de-Rosa - Los Estados Unidos del Fuego
Corona: The Killer
Coronavirus. Apocalypse
Country Mustard
Covid 19 Zombi Canibales del Infierno
Covid.12 the Hanged
Covid-19 Enquête sous confinement: Baudour
COVID-25 the Virus Has Mutated
COVID-25 the Virus Has Mutated
Cowgirls and Indians
CR: Complete Reality
Crazzy Lamhen
Creating the Cure
Creep Lodge
Creepy Ballz
Criminal Countdown
Crisis in London
Crisp Lips
Crooked Transit
Crossroads Siwoku
Crowdsourced Cinema VT: 

Ghost Cocaine: The Heist Before Christmas
Ghost Finder
Ghost in Transit
Ghost Story - An Abstract Film
Ghostface (Walshfilms)
Gialli II
Gidan Kashe Ahu
Gift Box
Ginga -Nagareboshi Gin- Gajo Kessen Hen (The Battle of the Stronghold)
Giorgio e Abdul cartoon show
Girl with Good Luck
Glanearagh
Glenn Wool: Viva Forever
Glia
Global Caper-A Self Taped Movie
Globos Negros
Gnana Gange
Go Fishin'
Go Iguanas! Mission Atlantis
Gods of No Mercy
Gol be khodi
Gold Touch Master
Gone: The Lost Boys of Belfast
Gongfu zongshi huo yuan jia
Gongsudo
Goo Is God
Good Girl Lola
Goodbye Julia
Goodbye to Trouble
Gookjesoosa
Gorbeh Siah
Gore Theatre 2
Goreyan Nu Daffa Karo 2
Graham's Dimension Travelers
Gramavrikshathile Kuyil
Grand Guignol madness - Show your fear
Grand kankan
Grand Theft Auto: MurderCity
Great God Monkey 1
Grilled Dream
Gudumana Avathara
Gujariya 2020
Gujjubhai 3
Gumdrop, a Short Horror
Gunfight at Silver Creek
Guns and Grams
Gymnastirio
Hâ luô shàonû: Girl's Revenge
Hétköznapi Trianon
Had Al

Lantai 4
L'appel de l'océan
Lara Tommasi - Il matrimonio
Las sirenas se van de viaje, una cine-performance de Sarah Benillouche
Last Chance Tango
Lato w Byszewach
Le Cineaste - A Director's Journey
Le Paradis des Sorcières
Le voleur rose
Leadfoot: LMack Racing
Learning Frankie Spanish
Left All Alone
LegacyVerse Productions' Twisted Christmas Carol
Legal Medical Expert Song Ci
Legendary Lover
Legends of the Heartland: The Life and Crimes of Return Ward, vol. 1
LEGO SCP Foundation: Containment Lockdown
Leichil Mafol
Lekki on Arrival
L'elefante nella stanza
Leo and the Two Fences
Les 3 Voeux
Les quatre filles fantastiques
Lethal Procedures
Let's Play: A Tale of Two Sisters
Letter of Reason
Lições de um Salvamento
Lian Qu 1980
Life Between the Reins
Life of Chris & Keisha (Animated)
Life on a Park Bench
Life when You Care
Like No Tomorrow: An Apocalypse Anthology
Like Reply Share
Lil Ceaser
Lil' Santa's Book Club: The Life and Adventures of Santa Claus Part 2
Lil Santa's Book Club: The New

Okeimadam
Oko Mi L'america
Ole Bryce
Olsen Banden på dupperne igen
Oma Ofe
Omaggio a Giordano Bruno (Il rogo, Il Candelaio, Il confusissimo secolo)
OMC Movie: The Movie
Ommomme
On Matriarchy Lake
On the Red Carpet: 41st Young Artist Awards
On the Waves of Faith
Once Upon a Time in Karachi
Once Upon a Time in Tollywood Vol. 1: Flop Director
Once upon a time in Yaropoltsy
Once Upon a Time in York Street
One Awful Summer
One Day of Freedom
One Night in China Town
One Place Forever
Online Guru
Onsen jôwa: Yubune de momi gaeri
Ooh myy Love
Operation Teddybear
Oracle's Spell
Orang-U 2: Monkey Business
Order 86
Orey Bujjigaa
Oru Pappadavada Premam
Oru Vadakkan Pennu
Ostfriesen auf der Bank - Der Film
Other Than Him
Our Communist Life
Our Gorgongosa
Our Jesus Story
Our Natural Right
Our Popcorn Movie Dystopia - Some More News: The Movie
Outside California
P.A.Carlmarks Baklängesyrket som gick framåt
Pánico Volumen 1
Paandiya Vamsam
Paapu
Pachamanga
Padauk Musical
Pagleaazam
Pal-Bot 2
Pange Yah

Shockfest Witch Trials Salem
Shrine Girl
Shrungaaram Art of Love
Shyamaragam
Si ai vaccini
Si j'étais fermier
Sihirli Annem Bir'iz
Silence Is Compliance
Silence of Sleep
Silent Notes
Simk Story Short Movies Theater
Simpallag Mathond Love Story
Sin Samejima Jiken
Single Ladies IV
Sinterklaas & de gouden chocolademunten
Sinthetic, the movie
Sir Sir Sarla
Siruthai Siva
Siskolar & Siskalar
Sisters of the Gion (Colorized Version)
Sister's Plot
Six: The Fully Animated Musical
Siyah Incinin Sirlari
Siyavar Ram - Ramayana
Skinny Bitches, Revenge of the Bears
Skinny Dipping
Skrivnosti z Beethovnove ulice
Sky High: Remote Learning
Slaindeer: Buck to the Future
Sleazy Slashers
Small Town Remedies
Smart Marks: The Movie
SMG4: Meggy's Destiny
Smithtown
Smokes and Jokes
Smokes and Jokes Atlanta
Snail Reads Hemingway
Snake Eyes, fan film
Sniping 2
Sobrevivir, sobremorir
Social Box
Sohayla Horani: Interested or Racist?
Sokku Sundaram
Somara
Someone2Luv: The Movie
Son of Mercy
Song ni yi duo xiao hong 

The Politician's Wife
The Poor Hoodman
The Power to Make a Change
The Prequel of Gold Convoyers: Struggle for Hosttages
The Prequel of Gold Convoyers: Super national Treasure
The Prodigal Cowboy
The Prophet of Mercy
The Purger
The Purser
The Puzzling Secret
The Rags of Time: J. Robert Oppenheimer
The Rapture Diaries
The Rarity Feature
The River Bank, the Couple, the Ghosts
The Road Less Cycled
The Road to Cabazon
The Rodfellows Movie
The Rotation of Love
The RoTfL podcast
The Saints of the Rue Scribe
The Sam's Day 2 - Alone at the Darkness
The Sand in Rime
The Savage Seconds
The Scarlet Pirate
The Scent of Betrayal
The Scorpion Monster
The Scrubbing Bench
The Scrubbing Bench
The Second Bed
The Secret Hitman
The Secrets That Bind
The Secrets That Bind
The Seven Sides of Shakespeare
The Sigurd Vedal Show
The Sin Choice
The Six of Us
The Sixth Land
The Sky Was Different
The Skyless Stars
The Spear of Destiny: The Passion of Saint Simon and Saint Jude
The Spider-Man (Fan Film)
The Springfi

Zuìran de quantou
Zui Ke Ai De Ren


In [100]:
df_2020['year'].value_counts().sort_index()

2020.0    9570
Name: year, dtype: int64

In [101]:
df_2020

,genres,originalTitle,year,director_name
13,Drama,1969,2020.0,Ernest Thompson
92,Drama Fantasy Sci-Fi,2050,2020.0,Princeton Holt
100,Mystery Sci-Fi,2067,2020.0,Seth Larney
223,Comedy,$30 Worth of Funny,2020.0,None
244,Comedy Romance Sci-Fi,(NE) Idealnyy muzhchina,2020.0,Marius Weisberg
...,...,...,...,...
68796,Adventure,Zorro: New Orleans,2020.0,None
68805,Drama Romance,Zovi menya Drozd,2020.0,Pavel Mirzoev
68817,Drama Thriller,Zu den Sternen,2020.0,Nicolai Tegeler
68818,Action,Zuìran de quantou,2020.0,None


In [103]:
df_2020.isna().sum()

genres              0
originalTitle       0
year                0
director_name    2949
dtype: int64

In [104]:
df_2020 = df_2020.dropna()
df_2020.isna().sum()

genres           0
originalTitle    0
year             0
director_name    0
dtype: int64

In [109]:
df_2020

,genres,originalTitle,year,director_name
13,Drama,1969,2020.0,Ernest Thompson
92,Drama Fantasy Sci-Fi,2050,2020.0,Princeton Holt
100,Mystery Sci-Fi,2067,2020.0,Seth Larney
244,Comedy Romance Sci-Fi,(NE) Idealnyy muzhchina,2020.0,Marius Weisberg
287,Drama Romance,...gia panta,2020.0,Don Roos
...,...,...,...,...
68695,Drama Fantasy Romance,Zona Seyl,2020.0,Scout Tafoya
68784,Horror,Zoomed,2020.0,Banmei Takahashi
68786,Drama Mystery,Zora,2020.0,Richard Pierrin
68805,Drama Romance,Zovi menya Drozd,2020.0,Pavel Mirzoev


In [105]:
df_2020.to_csv('./Dataset/processed_dataset/2020_dataset.csv',index=False)

In [106]:
df_imdb = pd.read_csv('./Dataset/processed_dataset/IMDB_dataframe.csv',sep=',')
df_imdb

,genres,originalTitle,year
0,Drama,1098,2018
1,Drama,1100,2019
2,Drama,1906,2019
3,Action Drama War,1917,2019
4,Horror Mystery Romance,1921,2018
...,...,...,...
56884,Drama Romance,Zwischen Sommer und Herbst,2018
56885,Drama Family,Zwischen uns,2021
56886,Drama Romance,Zwischen uns die Mauer,2019
56887,Action,Z-Wolf: A Yamcha Story,2022


In [110]:
df_2021 = df_imdb.loc[df_imdb.year == 2021,['genres', 'originalTitle','year']]
df_2021

,genres,originalTitle,year
11,Thriller,1980,2021
16,Comedy Romance,1990,2021
18,Fantasy Mystery Sci-Fi,1994,2021
19,Adventure,1997,2021
21,Drama Thriller,2024,2021
...,...,...,...
56867,Drama Romance,Zutto Dokushin de Iru Tsomori?,2021
56873,Comedy Romance,Zwaar verliefd! 2,2021
56876,Drama,Zwei ist eine gute Zahl,2021
56885,Drama Family,Zwischen uns,2021


In [111]:
df_2021 = df_2021.dropna()
df_2021['year'].value_counts().sort_index()

2021    11217
Name: year, dtype: int64

In [112]:
df_2021['director_name'] = df_2021['originalTitle'].map(lambda x: get_director(str(x)))

Aug-75
Jan-01
00K9: Operation Christmas
11: 59 the Online Play
12 Bulbule
12 palabras
12-kagetsu no Kai
13 Knives
17 Letters
1821: Anemos eleftherias
1945 Great War
1984: Theatrical play
2 Finesse
2 Minutes Wrong
2016 Reporter Robinhood Reporting
2020 compilation
20Banz 2
21 Mu Tiffin
2112 Bienvenida al Futuro
2nd Chances
30 Seconds in Hell
39 Ghosts
3am Waitress
40 & Climbing
49KG of Content
5 is Company
5egundos
72 Ghanta
72hrs the Hague: Outbreak
8 Beats Anthology Vol. 1
8119 Miles
90 Day Rule the movie
900 Guns
900 Guns
A Beautiful Request
A Billion Part Per Million
A Bloxy Movie
A Chat Movie
A Christmas Reckoning
A Cold Plan
A Cold Supper Behind Harrods
A Covid Christmas
A Day of Trouble
A Dose of Her Pill
A Faded Rose
A Fandemic: 50 Fans Celebrate 50 Years of Cinema
A Faraway House
A Fear of Moving Forward
A Fool and a Lover
A Fool for Trust
A Frozen Christmas Dance: Christmas Miracle
A Frozen Christmas Dance: Party Like A Reindeer
A Frozen Christmas Dance: Party Like A Turkey
A 

Broken Bottles
Broken But Whole
Broken Intentions
Broken Link
Broken Silence Family Edition
Broken Trust 2021
Broken Vine
Broken Walls
Brutal Decision
Bu lao qi shi
Bu su lai ke
Bu xiang yi ge ren
Bubujaan
Buck Alamo or (A Phantasmagorical Ballad)
Budynok Slovo. Neskinchenyi roman
Buka. Moyo lyubimoe chudishche
Bullet Satyam
Bully the Rico Story
Bungo2
Burn My Money
Buru- Hebun o Kimi ni
But I love him
Butta American 2021
By Any Means the Movie
Cühenna Nova Prospekt
Cabaret Cuba
Cachexia
Cacio E Papa
Cada uno tiene su cada uno
Caesar the Musical
Cafe in the Void
Cagliari - Juventus
Caillou and Penny
Call Me Elizabeth
Campfire Girl Massacre
Camping Paraíso** - About dying
Cam's Story
Captain Callum Explores the Universe
Captain Pineapple - Der Film
Caracol Guasu
Carl Jackson's LAX
Carmilla: Rise of a Vampire
Carter Family Reunion
Casanova B - All you need i$ Lov(e)a
Case No 130/2016
Cast Out
Catch Out
Cawdor Street Thanes
Ce huang ren
Ce qui reste passé minuit
Ceberruh
Celebrity Girlfri

English Padha Na Balamua
Enlace de Muerte
Enna: Kara Büyü
Ennathaan Un Kathai
Enough-Story of Lucy
Enquanto o Céu Não Me Espera
Equilibrio degli opposti
Esìn
Escape from New York: The Official Story of the Film
Esquadrão na Moral
Esther & Maude
Eswarecha
Eucharisztia
Eureka/Kôkyô Shihen Eureka Seven Hi-Evolution
European Championships Final
Eva Garner the Secret of Eden
Evadra Hero
Everlasting Dream
Everyday Black Matter
Everything But a Ring
Evil Dead Is Back
Evil Forest Horror Film
Ewon Ife
Examining Eloise: A case study in mental illness and demonic influence
Exchange Eternal
Expediente 53
Expediente 53
Expedition Unfeasible
Exploración Mortal
Extra Mile
Extra Pen
Extremophile
Eyan Ni Mi
Fa cai ri ji
Fâsuto ravu
Fabric Unraveled
Face of Greed
Faceblocked
Faces of the Dead 2: Exorcism
Facundo Battocchio's Aislados
Falling in Slow Motion: Proof of Concept
Faltu Lolli
Fanan
Fanchu
Fang Shiyu the Winner Is King
Fantaisies et chroniques
Fantasmagorie - The Ghost Show
Far Away Form Here
F

Jiao tou wai zhuan: Lang liu lian
Jihad - New Life
Jijau
Jimmedari
Jin's Emperor
Jirah and the Road Out of Copperhead
Jivan Aakhyan
Jo & the Reaper
JO5H AKER5 FILM5: 5 years later
Joel D. Wynkoop's the Craiglon Incident
Joel D. Wynkoop's the Craiglon Incident II Aftermath
Joexian and Nate: The Movie
John from Home
John Neptune and the Sky Palace King
Johnnie & Glyde
Johnny in Monsterland - The Directors Cut
Joker: Part II
Jolghori - Story Never Dies
Jonny Loquasto: Physical. Therapy.
Josh Hurts
Joshua's Seance
JtK
Judaseffekten
Juese taosheng
Julius Caesar: Beware the Ides of March of the Penguins
Jurassic Planet: The Mighty Kingdom
Just South of Insanity
Justice For Good Content
Justice League 1995
K K Hundred
Kôkaku kidôtai SAC_2045 Jizoku kanô sensô
Kaage Motte
Kaali Maati
Kaanagasattam
Kaanbhatt
Kachufool
Kadaisi Kadhal Kadhai
Kadal Paranja Katha
Kaddu Muchchi
Kadhanika
Kadhi Ambat Kadhi Goad
Kagayaku nareba sore
Kaguya-sama wa kokurasetai 2: Tensai-tachi no ren'ai zunôsen final
Ka

Midsummer Night's Scheme
Midunu Vishwaya
Mientras Me Abraces
Migraine Documentary
Migrant Birds Fly
Mike Lebovitz: Two Slob Household
Mikha's Summer
Milagro de Luna
Milano Debbie
Milk It: Lyrical Dance Fusion
Millennial Zindagi
Millennium Hope Hills
Millions of Cards
Milyonda Bir
Mimi's Voice
Minchuhulu
Mind Dates
Minecraft: Super-Heróis 4 - O Filme
Ming tian hui hao de
Minibus Driver
Minor Daemon, Volume 1
MIRA Supernatural Assassin
Mise en Place: Conquering the Marsala
Mishak Hachvaim
Misplaced Fortune
Miss Koala's Class: Learn About Animals
Miss Shaguna's Chickens
Mission Concepción
Missiya: Prorok
Missperception
Missunderstanding
Mister Mayfair
Mi-syeon pa-seo-beul
Mitabime no, shojiki
Mithu Singh Da Viah
Mme l'ambassadrice
Modern Roomies
Modernity Meets Crafts
Modhala Minchu
Mofe Ni Mofe
Mohadaa
Mole to Society
Mommune
Mommy Lounge Show Podcast Movie
Mon insolence
Monday Osunbor
Monks of Babylon
Monopoly Money: The Movie
Monotonic Melvin and the Midnight Maelstrom
Monster from Bri

Promised: A True Story
Prophetic Prayers
Proyecto Higuera
Psiico
Psycho Varma
Pticar
Pueblo Chico Invierno Grande
Puli Vachindi Meka Chachindi
Pulpit Gangster: Life Inside a Cult
Punashchha Hari Om
Punk Rock Cannibals
Pup School Jr: Fun with Numbers
Pup School Jr: How Are We Made
Pup-o, ma2! The crazy bride
Puppy of a Bitch
Puppy-Pedia the Dog Encyclopedia: German Shepherds
PursotRama
Pururava
Push Comes to Shovel, TX
Pyar Kiya Toh Nibhana
Python of Jungle
Qian qing cheng bi de shi dai
Qing chun shi lian
Qualcos'altro che ancora non c'è
Quand la mer séduit la montagne
Quarantine Bae
Quarantine, Actually
Que le Père soit
Queen Lateefat
Queens Keep
Quel genio del mio amico
Quest for Steel
Questhaven
Questions and Comments
Quick Wisdom with Bliss: Mantra
Quick Wisdom with Bliss: The History of Yoga
Quit Notice
R A N D, inc.
R.E.M. Taheies kiniseis ofthalmou
Raave Naa Cheliya
Rabbids Escape
Radhe ki Dulhaniya
Radiant Dark
Ragazze dei fiori
Raghav Radio
Raghu in Mumbai
RAHMAH: Silent Voices

Stopped Motion: The Story of GULP
Story of Dinda: The Second Chance of Happiness
Story of the third world
Stranded Astronaut
Strange Fruit: Tale of a Black Girl Lost
Strange Troubles: Ghost of Ebube
Strawberry List
Streets of Philly Undercover: Chapter 1
Stretched Heart
Strut That Ass!
Stryi. Legend
Subconscious III: The Final Chapter
Subject 18
Succubi Unbound!
Suddenly: A Short New Musical
Sui yue hu yi mu
Suicidio Apparente
Sukh Mhanje Nakki Kaay Asta
Sukhparijat
Sukutte goran
Summer in Hindsight
Summer Sonatina
Sunday Ewenje
Sunday Jatto
Sunshine Circle
Supadu Dada: The Great
Super Battle Junretsuger
Super Duper Heist Squad
Super Grannies
Super Hijabi
Super Kudalar II
Super Mario Into the Literal Triangle - A Mario Plush Movie
Super Moon Ogikubotakashinopureibaku
Super Vixens 2
Super Vixens 3
Superblocked
Superman: Mortal Menaces
Supermind: The Brain Drain
Sup-eui bi-myeong: sa-ra-jin sa-ram-deul
Supilreobeu
Surge: Chapter 1 - Hydrocene
Suricatos Albinos
Suryasthamayam
Susaekja
Swa

The Stacey Stacey Story: Hell in My Life
The Story of Nehanda
The Story of Shanti
The Story of the Bloody Hand of the Six Doors
The Straight Path
The Strange Demise of Reginald Thatcham
The Sweded Film Festival for Creative Re-creations
The Symphony of Pansies
The Tag Conspiracy
The Teen Commandments
The Ten Thousand Sword Seal Demon of Shu Mountain
The Thing That Keeps You
The Thing: O Regresso
The Things We Bloom
The Thot Who Stole Christmas
The Three Brides
The Three Little Pigs Storytime
The Tremont
The Truck Driver Movie
The True Blue Ripper
The Twelve Step Strangler
The two emperor poets
The Two Lucys
The Ultimate Chess Move
The Ultimate Riddle
The Unbreakable Marriage
The Undertaker's Wife
The United Kingdom of Horror
The Valley Monologues
The Vanished Voice
The Vitruvian Theory
The Voice Remains
The Wages of Sin by Michael Kleos
The Wasteland of Education
The Wave of Mega Shark!
The White Vault: Avrum
The Witch's Pet
The Woods That Made Me
Theatre Gods: A Virtual Comedy
Theoret

Zuk seoi piu lau
Zunjar Gadya
ZURI - A Rare Soul
Zwtral


In [113]:
df_2021

,genres,originalTitle,year,director_name
11,Thriller,1980,2021,Keralino Sandorovich
16,Comedy Romance,1990,2021,Nhất Trung
18,Fantasy Mystery Sci-Fi,1994,2021,Adem Suljic
19,Adventure,1997,2021,Dr. Mohan
21,Drama Thriller,2024,2021,Rohin Raveendran
...,...,...,...,...
56867,Drama Romance,Zutto Dokushin de Iru Tsomori?,2021,Momoko Fukuda
56873,Comedy Romance,Zwaar verliefd! 2,2021,Marc Willard
56876,Drama,Zwei ist eine gute Zahl,2021,Stefan Hering Holger Borggrefe
56885,Drama Family,Zwischen uns,2021,Max Fey


In [114]:
df_2021['year'].value_counts().sort_index()

2021    11217
Name: year, dtype: int64

In [115]:
df_2021.isna().sum()

genres              0
originalTitle       0
year                0
director_name    3538
dtype: int64

In [116]:
df_2021 = df_2021.dropna()
df_2021.isna().sum()

genres           0
originalTitle    0
year             0
director_name    0
dtype: int64

In [117]:
df_2021

,genres,originalTitle,year,director_name
11,Thriller,1980,2021,Keralino Sandorovich
16,Comedy Romance,1990,2021,Nhất Trung
18,Fantasy Mystery Sci-Fi,1994,2021,Adem Suljic
19,Adventure,1997,2021,Dr. Mohan
21,Drama Thriller,2024,2021,Rohin Raveendran
...,...,...,...,...
56865,Comedy Drama,Zussen,2021,Thijs Zeeman Manon Hoornstra
56867,Drama Romance,Zutto Dokushin de Iru Tsomori?,2021,Momoko Fukuda
56873,Comedy Romance,Zwaar verliefd! 2,2021,Marc Willard
56876,Drama,Zwei ist eine gute Zahl,2021,Stefan Hering Holger Borggrefe


In [118]:
df_2021.to_csv('./Dataset/processed_dataset/2021_dataset.csv',index=False)

In [119]:
df_imdb = pd.read_csv('./Dataset/processed_dataset/IMDB_dataframe.csv',sep=',')
df_imdb

,genres,originalTitle,year
0,Drama,1098,2018
1,Drama,1100,2019
2,Drama,1906,2019
3,Action Drama War,1917,2019
4,Horror Mystery Romance,1921,2018
...,...,...,...
56884,Drama Romance,Zwischen Sommer und Herbst,2018
56885,Drama Family,Zwischen uns,2021
56886,Drama Romance,Zwischen uns die Mauer,2019
56887,Action,Z-Wolf: A Yamcha Story,2022


In [120]:
df_2022 = df_imdb.loc[df_imdb.year == 2022,['genres', 'originalTitle','year']]
df_2022

,genres,originalTitle,year
5,Action Drama History,1945,2022
7,Drama,1960,2022
10,Drama,1976,2022
22,Drama,2037,2022
29,Drama,4554,2022
...,...,...,...
56871,Drama,Zwölferleitn,2022
56874,Comedy Romance,Zwanger & co,2022
56880,Thriller,Zwielichts Pilgrim,2022
56881,Drama,Zwigato,2022


In [121]:
df_2022 = df_2022.dropna()
df_2022['year'].value_counts().sort_index()

2022    12795
Name: year, dtype: int64

In [122]:
df_2022['director_name'] = df_2022['originalTitle'].map(lambda x: get_director(str(x)))

4554
Jun-45
0002: A Sharkventure
07.28.1999
1 Out of 100
10 Tricks
100 Crores
100 Dresses
1000 deshevykh zazhigalok
100th Anniversary Riff of Nosferatu
12 Ghost Fingers
14 days Love
1600 Miles
1915: Legend of the Gurkhas
1936: The Abdication
1948 Akhand Bharat
1965 Underworld Rising
20 Ways to die alone
2020 Golmaal
2021 La Peste
20segi sonyeo
21 Divas
2323 the Beginning
24 Hours A Sweet Kiss Goodnight
24 Hours Hot Pursuit
24 hours in Gethsemane
2G Apartments
2nd February
3 Can Play
3 Days Till Darkness
3 Dudes
3.3 Miles
3: An Abstract Film About Identity
30 Days No Sleep
35th of May
36 Gunn
38,5 quai des orfevres
3Devi
3lovers
41 Kere Masallah
42-50Nirvana
45 Mill Street
45 Minutes from Edith Wharton
48 Ghante: Do Dil Do Raat
5 Days to New Year
5 Seasons - eine Reise
50, boulevard Voltaire
52 Days: The Triangle
59 Minutes in a Bunker with the President of the United States
5il Oral Thaskaran
6 Wheels from Hell!
666 Derry Ln
69 Parts
6-Crowd
7Even Hits
7th & Westlake the Rerock
8 Bucks

Best Friends Forever: True friends
Bestie da Macello 2 - La vendetta dei fratelli Sparvieri
Beton Nazmiye
Bettada Dari
Between Sins
Between Two Hearts
Beurokeo
Beware Deth Is Coming
Beyond the 7 Seas
Beyond the Grave: Unforgotten Voices
Beyond the North Woods
Beyond the Speed of Life
Beyond the White Veil
B'ezrat HaShem
BFDI Plush: A Trip to Las Vegas
BFDI Plush: The Big Vacation
Bhaani
Bhala Chora Bhala
Bhala Chora Bhala
Bharat Mazha Desh Aahe
Bharateeyans
BhauBali
Bheerya
Bholaa Bhaai
Bhoonataka Mandali
Bhoot Uncle Tusi Great Ho
Bhoy - The Beginning
Bhraman Dhwani
Bi-Coastal
Bienvenido a tu Vida
Bihe Pass
Bilbo and the Legend of the Last Living Cow
Billu V.C.R
Billy Smith
Bin yun haang ze
Binky Petunia
Binny and Baba
Bir Kar Tanesinin Ömrü
Bir Türk Masali
Bird Scouts: Learn About Birds
Birthing Pains
Bitcoin Baby
Biye Retour
Black Ambrosija Blossom
Black and Blue Tears
Black Girl Magic Las Vegas
Black Nazi-Hunter
Black White and the Greys
Blackstone - Hand of God
Blackwater Blues
Bla

Diane Spencer: Slid Off the Turkey
Diario de cuarentona
Diary of the Underdogs
Die Hard Fan
Diflood7
Digital Asylum
Dil Dimag Aur Batti
Dimensions 5
Din dzi ling
Ding Dong Witch Dead
Dinner for Promotion
Direct Son
Dirty Mirrors
Dirty Reflections
Disc Golf Land
Discerning Eyes
Discovered: The Director's Cut
Dis-topia
District Queens: The Racine Robinson Story
Divine Telepathy
Divya'Drushti
DJ Khaled: Another Win
Django JD
Do Not Let Your Dragon Spread Germs
Do We Call It Love?
Doc Resureccion: Gagamutin ang Bayan
Doce Encanto
Doctor Bitcoin Makes the Magic
Doctor Cengiz: Cigiliverse of Madness
Doctor Strange: Multiversal Madness
Doctor Who: Mudlarks
Dokunma Hurdacinin Kizi
Dole Mates
Dombari
Don Bhaisahab
Don Macario Leyva
Donde Habitan Los Secretos
Done Waiting: Chapter 2
Dong-e beonjjeog seoe beonjjeog
Donkey Dust
Don't Break My Home
Don't Call Me God of Gamblers
Don't Cry for Bombardier
Don't Lick the Iguana
Don't Shoot the Piano Man
Don't Tear Yourself Apart
Don't Test My Love
Doom

Grim Reaper 2
Grinding Humanity
Grisha Subbotin
Gritty Romance
Guan yu wo han gui bian cheng jia ren de na jian shi
Guarded Heart
Guardian of Aegora
Guddobai, baddomagajinzu
GUHAAR
Gui de fang
Guide A Journey into Heart
Gulhar
Gumball 3000 - The Last Mile Home
Gumball Dreams
Gummy Worm
Gun Camera Footage: Desert Storm Declassified
Gunfight at Deadman Wash
Guntur Talkies 2
Gunz N Roses
Gurbat
Guwahati Diaries
GV Il dio della moda
Gwo si gwo dzit
Gye Nyame
Gyoto-eseo on pyeonji
H.B.S.U? Historically Black Sellout U?
Héritage fatal 2
Habiba un amour en confinement
Habsora al pi Yehuda
Hachigatsu wa nigete hashiru
Hadal
Haelwen-Sian: Waiting for My Love
Hafir: Karanlik Mezar
Hai de jin tou shi cao yuan
Hai ou lai guo de fang jian
Haint Blue
Halakhah
Halli Haikla Pyate Lifu
Halloween: Legacy of the Witch
Halloween: O Legado
Hallowtide
Hama guri sankei: Sutte ijitte
Hamlet rock-musikaali
Hammer Home
Hand Vampire
Hanging Up with Mater
Hankuri
HAnnimal Lector
Hanuman Mantras
Happy Days - La ve

Korku Takvimi
Koshichaayante Paramb
Kothe Pakke Rishty Kache
Kothhi 1947
Koutilya
Krishna Sangani Yamuna
Krooked Lines
Kshanikam
Kshipra
Kuhok
Kulche Chole
Kulswamini
Kumara Sambhavam 2022
Kundi Na Kharka
Kungfu Master Su: Red Lotus
KunLun Battle
Kuqui Alegre feat. Playa Cuberris: Quinquis
Kuvah
Kya Rakha Hai Naam Mein Main Nahin Jaanta Magar Mera Naam Ullhas Hai Mera Parivar Aur Mere Jaanane
Kya rakha hai naam mein, main nahin jaanta magar, mera naam ullhas hai, mera parivar aur mere jaanane wale muze isi naam se jaante hain
L.A.C.
L: Life is Learning
La balada de los cuervos
La banda degli asini
La caja y el valor de la libertad
La Canasta de Huevos
La Canasta de Huevos 2
La celeste inquietud de las cosas
La Certitude du Saumon
La conquista de un recluso
La cygne des héros
La Dame Des Ténèbres
La fleur du rocher
La gare de l'est
La Giaguaro
La Hija de la Muerte
La Jefa De La Plaza
La Jom Atenda
La loca del maniquí
La lunga notte dei morti
La Maison de la Vanille
La Mujer de Vidrio
La

My Babysitter the Super Hero
My Boomerang
My Brain at 11:37pm: Visceral Visions
My Brother Cain
My Constant Shadow
My Drowning Man
My Father the Dog
My Friend Max: Friends Forever
My Home (Ile Mi)
My Last Best Friend
My Lithuanian Dream
My Name Is Heather
My Name Is Lama
My Name Is Lucien
My Name Is Noah and This Is My Fucking Ark
My Name Is Tayo
My Patient and I
My Perfect Love
My Ripped Off Movie
My Shattered Life
My Special Boy
My Summer in Ohio
My Sweet Next Day
My Wife, My Trouble
myPhone
Myself Clement
Na Bhayam Na Lajja
Na Ribas
Na venta paduthunna chinnadevadamma
Na vydokhe
Nü zi te bie xing dong
Naach Baiju Naach
Naadhiru Dhinna
Naalo Ninnu Dachaane
Naam Zubaan Pe Tera
Naan Aval Ilaiyaraaja
Naanu Adu Mattu Saroja
Nabil El Gamil Dr. Tagmeel
Nadeeya
Nadie te Recordará
Nahi Jnanena Sadrusham
Naihar
Naina ki Sharafat
Nainaa
Najsrekjniot chovek na svetot
Nalapani
Nalwawo
Namak Haraam (2022)
Namastey Ji, May I come in
Namma Hudugaru
Namur
Nan Jothe Poojalakshmi
Nanabis
Nanban Oruvan

Raghuram (Bhojpuri)
Raiders of the Golden Bear
Rain Falls on a Blossom Tree
Raisin Paste
Raja rani roarer rocket
Rajahmundry Rose Milk
Rajnandini
Rajneeti - Part 1
Rajpal
Ram Mohan Kanchukommala
Rama: Gau Ka Gangster
Ramayan (Motzoid)
Ramrajya
Ramudu Anukoledhu Jaanaki Kalaganaledhu
Ran Mi Lowo (Help Me)
Randam Mukham
Rangbaaz Wanted
Rangdaar Shiva
Rangilo Rasiklal
Ranking All 836 Moons in Super Mario Odyssey
Rankolla
Río Grande, lo que el viento no arranca lo arraiga
Ríos de Ceniza
Raquel 1,1
Rasmus geht seinen Weg
Raspberry1508
Rasyah the Wonderkid
Rat on a Highway
Ratt's Life
Razvod. Film vtoroy
RCD Mallorca contra Cádiz CF
Real Connections
Real Emperor Kandanai Maneesawath
Real Madrid - Valencia CF
Really Main Deewana Ho Gaya
Reanimation Team
Reús 2
Rearick Film Studios' Doctor Mobius
Reasons and Endings
Rebellion USA Redux
RECCE 360
Rechipodam Brother
Red Bluff 2: For Human's Sake
Red Forest Khooni Jungle
Red Horse in the Summer Sky
Red Raincoats
Red Under Wraps
Reddy Garintlo Row

Suraapanam
Surabhi 70mm
Surrounded by Idiots
Svidanie v Minske
Svjedoci proslog vremena
Svoya voyna. Shtorm v pustyne
Swacha Karnataka
Swap Life
SWAP(PARO)
Swapnangal Pookkunna Kaadu
Swapnasundari
Swaraj: Bharat Ke Swatantrata Sangram Ki Samagra Gatha
Swash Buckler & The Plumed Sorceress
Swathi Chinuku Sandhya Velalo
Swayamsevak
Sweet BodyGuard
Syd and Chloe and the Myriad Worlds
Sylas: The Last Gunman
Sylvester Both Sides
Ta anthi sta anthi
Tóng huà shì jiè
Ta'am al Tufah, Ahmar
Tabaahi Reloaded
Tablet moments, built by you!
Tad Caldwell & The Monster Kid
Tahit Tahdid Alsilah
Tainted Heart
Tajmahal 2
Takatak 2
Take me to Amazing (TMTA)
Takhatgarh
Tales from Apocalypse
Tales from Pandora's Box: Chapter One
Tales from the Creek
Tales from the Crip 2
Tallika
Tamasha Live
Tangled Sing-Along
Tania's Trap
Tantum
Tao xue shen tan
Tapana
Tar Galpo
Taraf Talet
Task Force: Down the Rabbit Hole
Tasveer Ishq Ki
Tchiza titulaire
Tea Street
Teaching Faith
Tear Away
Teen Titans: Alpha
Teenage Warhea

The Things I Have Seen
The Thot of Murder
The Thracian Curse
The Thursday Night Club
The Traveler: With the Voice of Peter Baykov
The Truth Is a Foot
The Twisted Ones
The Two Diaries (Part 1)
The Ulysses Project
The United Nations of Horror
The Unsolved Kidnapping of Madeleine McCann
The Uprising 4910
The Vale Untold
The Value of Nothing
The Vampire and the Vigilante
The Wage of Better Angels: A Treatise to the Truth
The Washington County Disappearances
The Way of Generations
The Weatherman Chronicles
The Wise One
The Wisteria Manor
The Wizard 20Twenty
The Wizards Books: A Tale of Three Sisters
The Woman Under the Bed
The Workshop, A Dress Rehearsal for Life
The World Needs A New Superhero
The World of Kapata
The Worst of Faces of Dying
The ZackFilmz Movie
Theater of the Supernatural: City of the Dead
Theoretically, an even much more paranoid conspiratorial phone call: The Thespian
Things We Do in the Shadow
Tholi grammi
Thomas & Bertie: The Major Race
Thomas A Peeper
Thopukaranam
Thor

Yellow Weekends
Yemaipothaney
Yevaal
Yi bian bao long
Yi Chang hen (Mei) You Bi Yao De Chun Wan
Yi jia zhi zhu
Yi jia zi er gu gu jiao
Yi tin to lung gei
Yi tin to lung gei 2
Yi zhi yi zhi dou hen xi huan ni
Yi zhou de peng you
Yildizlara Fisilda
Yin ru chen yan
Yin Yang Zhen guai tan
Yoake made bus tei de
Yolki-igolki
Yolo County O.G
Yong bu tuo xie
Yoru Toritachiga Naku
Yoshi Life: Squid Days
You and What Army
You Bring It Back with You
You Don't Deserve This
You Married Dat
You Who I Called Sister
Youforia
Youkai Share-house -Hakuba no oujisama ja nainkai-
Young Bloods - Concept Film
Young Dario
Young George 2 No Dragons Allowed!
Your Luck Today
Youth Festival 2022
Yu zhou ji yi
Yudi: Yedikonuk Cinleri
Yue jie
Yunque Forest: Area 51
Z+ A Deadly Case of Senioritis
Zaaklou meican
Zabytoye chudo
Zavrashtane 2
Zebun
Zehirli Tohumlar
Zehsin Cin-i Musallat
Zendaman 2
Zeréck
Zero Tolerance: A Cyberpunk Universe - Upgraded
Zhanna
Zhe ge sha shou bu tai leng jing
Zhen Dao Ge Zhi Chen Shi Xin

In [123]:
df_2022

,genres,originalTitle,year,director_name
5,Action Drama History,1945,2022,Ferenc Török
7,Drama,1960,2022,Gabriele Salvatores
10,Drama,1976,2022,Manuela Martelli
22,Drama,2037,2022,Enric Pardo
29,Drama,4554,2022,None
...,...,...,...,...
56871,Drama,Zwölferleitn,2022,None
56874,Comedy Romance,Zwanger & co,2022,Johan Nijenhuis
56880,Thriller,Zwielichts Pilgrim,2022,Joshua S. Bischof
56881,Drama,Zwigato,2022,Nandita Das


In [124]:
df_2022['year'].value_counts().sort_index()

2022    12795
Name: year, dtype: int64

In [125]:
df_2022.isna().sum()

genres              0
originalTitle       0
year                0
director_name    4092
dtype: int64

In [126]:
df_2022 = df_2022.dropna()
df_2022.isna().sum()

genres           0
originalTitle    0
year             0
director_name    0
dtype: int64

In [127]:
df_2022.to_csv('./Dataset/processed_dataset/2022_dataset.csv',index=False)

# Unifing all of them 

In [129]:
df_2018 = pd.read_csv('./Dataset/processed_dataset/df_2018_dataset.csv',sep=',')
df_2019 = pd.read_csv('./Dataset/processed_dataset/2019_dataset.csv',sep=',')
df_2020 = pd.read_csv('./Dataset/processed_dataset/2020_dataset.csv',sep=',')
df_2021 = pd.read_csv('./Dataset/processed_dataset/2021_dataset.csv',sep=',')
df_2022 = pd.read_csv('./Dataset/processed_dataset/2022_dataset.csv',sep=',')

In [130]:
final_df = df_2018.append(df_2019,ignore_index=True)
final_df.shape

C:\Users\rafae\AppData\Local\Temp\ipykernel_7128\2647580953.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df = df_2018.append(df_2019,ignore_index=True)


(29483, 4)

In [131]:
final_df = final_df.append(df_2020,ignore_index=True)
final_df.shape

C:\Users\rafae\AppData\Local\Temp\ipykernel_7128\142093142.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df = final_df.append(df_2020,ignore_index=True)


(36104, 4)

In [132]:
final_df = final_df.append(df_2021,ignore_index=True)
final_df.shape

C:\Users\rafae\AppData\Local\Temp\ipykernel_7128\3429659863.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df = final_df.append(df_2021,ignore_index=True)


(43783, 4)

In [133]:
final_df = final_df.append(df_2022,ignore_index=True)
final_df.shape

C:\Users\rafae\AppData\Local\Temp\ipykernel_7128\2440229342.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df = final_df.append(df_2022,ignore_index=True)


(52486, 4)

In [138]:
final_df['year'].value_counts().sort_index()

2018.0    11677
2019.0    11666
2020.0     6585
2021.0     7628
2022.0     8662
Name: year, dtype: int64

In [137]:
final_df = final_df.dropna()
final_df.isna().sum()

genres           0
originalTitle    0
year             0
director_name    0
dtype: int64

In [153]:
final_df.to_csv('./Dataset/processed_dataset/main_data.csv',index=False)

In [158]:
final_df = pd.read_csv('./Dataset/processed_dataset/main_data.csv',sep=',')

In [159]:
final_df['comb'] = final_df['director_name'] +' ' + final_df['genres']

In [161]:
final_df

,genres,movie_title,director_name,comb
0,Drama,The Other Side of the Wind,Orson Welles,Orson Welles Drama
1,Comedy,A Thin Life,Jay Jalbert Kenny Bernardini,Jay Jalbert Kenny Bernardini Comedy
2,Drama,Reverse Heaven,Stuart Paul,Stuart Paul Drama
3,Action Drama,To Chase a Million,Pat Jackson,Pat Jackson Action Drama
4,Drama Family,Aamaar Bhuvan,Mrinal Sen,Mrinal Sen Drama Family
...,...,...,...,...
91151,Horror,the burkittsville 7,Ben Rock,Ben Rock Horror
91152,Sci-Fi,caged heat 3000,Aaron Osborne,Aaron Osborne Sci-Fi
91153,Drama Action Romance,robin hood,John Irvin,John Irvin Drama Action Romance
91154,Drama,century of birthing,Lav Diaz,Lav Diaz Drama


## Combining the dataFrame from this file with the one with the other that contains move from before 2018

In [163]:
df_second = pd.read_csv('./Dataset/processed_dataset/second_dataframe.csv',sep=',')

In [164]:
final_df = final_df.append(df_second,ignore_index=True)

C:\Users\rafae\AppData\Local\Temp\ipykernel_7128\4275946701.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df = final_df.append(df_second,ignore_index=True)


In [169]:
final_df.drop_duplicates(subset ="movie_title", keep = 'last', inplace = True)

In [171]:
final_df

,genres,movie_title,director_name,comb
0,Drama,The Other Side of the Wind,Orson Welles,Orson Welles Drama
1,Comedy,A Thin Life,Jay Jalbert Kenny Bernardini,Jay Jalbert Kenny Bernardini Comedy
2,Drama,Reverse Heaven,Stuart Paul,Stuart Paul Drama
3,Action Drama,To Chase a Million,Pat Jackson,Pat Jackson Action Drama
4,Drama Family,Aamaar Bhuvan,Mrinal Sen,Mrinal Sen Drama Family
...,...,...,...,...
136089,Horror,the burkittsville 7,Ben Rock,Ben Rock Horror
136090,Sci-Fi,caged heat 3000,Aaron Osborne,Aaron Osborne Sci-Fi
136091,Drama Action Romance,robin hood,John Irvin,John Irvin Drama Action Romance
136092,Drama,century of birthing,Lav Diaz,Lav Diaz Drama


In [172]:
final_df.to_csv('./Dataset/processed_dataset/main_data.csv',index=False)